# Intersectar buffers con radios y fracciones censales

Los datos del Censo Nacional 2010 están publicados en forma de radios y fracciones censales. Cada radio o fracción censal puede caer dentro de muchos buffers a la vez. Para componer indicadores publicados a nivel de radio censal en indicadores por buffer, cada buffer tiene que "saber" qué radios y fracciones censales tiene dentro suyo y en qué porcentaje de superficie.

El supuesto a utilizar a la hora de componer indicadores agregados será el de que la población (tanto en cantidad como en características) se distribuye homogéneamente dentro del radio o fracción. Posteriormente se podrá comparar el resultado entre la composición por radio y por fracción, para chequear si la utilización de la menor unidad de desagregación (el radio) representa una mejora estadísticamente significativa respecto de la utilización de fracciones (que son unidades más grandes, donde uno debería pensar que el supuesto de homogeneidad es más peligroso).

In [1]:
from scripts import calculate_weights
from scripts import geo_utils
from scripts.path_finders import get_division_path, get_weights_path, get_division_dir
import shapefile

In [2]:
fracciones_dir = "shp/divisiones/fracciones_caba_censo_2010"
fracciones_path = "shp/divisiones/fracciones_caba_censo_2010/fracciones_censo_2010"
subte_buffer500_path = "shp/transporte/buffers/estaciones_de_subte-buffer500/estaciones_de_subte-buffer500"

In [3]:
# para hacer pruebas
sf_fracciones = shapefile.Reader(fracciones_path)
fracciones = geo_utils.get_shapely_shapes(sf_fracciones)

sf_subte500 = shapefile.Reader(subte_buffer500_path)
estaciones = geo_utils.get_shapely_shapes(sf_subte500)

In [4]:
print "Cantidad de fracciones censales", len(sf_fracciones.records()), len(fracciones)
print "Cantidad de estaciones de subte", len(estaciones)
print "Campos disponibles en el shp de estaciones de subte:",
for field in sf_subte500.fields[1:]:
    print field[0],

Cantidad de fracciones censales 351 351
Cantidad de estaciones de subte 83
Campos disponibles en el shp de estaciones de subte: ID ESTACION LINEA


Primero calculamos las intersecciones de radios censales con barrios, ya que los indicadores del Censo Nacional no se expresan por barrio de la CABA. El principio utilizado será el mismo para los buffers.

In [5]:
radios_dir = get_division_dir("radios_censo_2010")
fracciones_dir = get_division_dir("fracciones_censo_2010")
# Atención! el shp original de barrios tiene problemas de encoding, si se redescarga hay que corregir "Nuñez" en el dbf
barrios_shp_path = get_division_path("barrios_censo_2010")
empty_shps_dirs = calculate_weights.EMPTY_SHPS_DIR

Calculating intersections between radios_censo_2010 and estaciones_de_subte-buffer1000
Calculating intersections between radios_censo_2010 and estaciones_de_subte-buffer750
Calculating intersections between radios_censo_2010 and estaciones_de_subte-buffer1500


In [7]:
barrios_weights_radios = calculate_weights.calculate_intersect_weights(radios_dir, barrios_shp_path, 
                                                                       get_weights_path(barrios_shp_path, "radios"))

Calculating intersections between radios_censo_2010 and barrios_censo_2010


In [7]:
#fracciones_dir = get_division_dir("fracciones_caba_censo_2010")
#barrios_weights_fracc = calculate_weights.calculate_intersect_weights(fracciones_dir, barrios_shp_path)
#calculate_weights.save_to_json(barrios_weights_fracc, get_weights_path(barrios_shp_path, "fracciones")) 

In [8]:
# para calcular los % de intersección de todos los buffers
# esto puede tomar bastante tiempo si se recalculan todos... (varias horas)
calculate_weights.main(recalculate=False)